<a href="https://colab.research.google.com/github/teju163dst/Customized-Sentiment-Analysis-using-Hugging-face-dataset/blob/main/Sentiment_analysis_using_Hugging_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install libraries
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 98.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.7 MB/s eta 0:00:00


In [2]:
# Data processing
import pandas as pd
import numpy as np

# Train test split
from sklearn.model_selection import train_test_split

# Modeling
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# Hugging Face Dataset
from datasets import Dataset

# Import accuracy_score to check performance
from sklearn.metrics import accuracy_score

In [3]:
# Read in data
amz_review = pd.read_csv('amazon_cells_labelled.txt', sep='\t', names=['review', 'label'])

# Take a look at the data
amz_review.head()

,review,label
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [4]:
# Get the dataset information
amz_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  1000 non-null   object
 1   label   1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [5]:
# Check the label distribution
amz_review['label'].value_counts()

0    500
1    500
Name: label, dtype: int64

The label value of 0 represents negative reviews and the label value of 1 represents positive reviews. The dataset has 500 positive reviews and 500 negative reviews. It is well-balanced, so we can use  accuracy as the metric to evaluate the model performance.

## Perform Train test split


In [6]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(amz_review['review'],
                                                    amz_review['label'],
                                                    test_size = 0.20,
                                                    random_state = 42)

# Check the number of records in training and testing dataset.
print(f'The training dataset has {len(X_train)} records.')
print(f'The testing dataset has {len(X_test)} records.')

The training dataset has 800 records.
The testing dataset has 200 records.


Lets Tokenize the text data
A tokenizer converts text into numbers to use as the input of the NLP (Natural Language Processing) models. Each number represents a token, which can be a word, part of a word, punctuation, or special tokens.
* `AutoTokenizer.from_pretrained("bert-base-cased")` downloads vocabulary from the pretrained `bert-base-cased` model.
* `return_tensors="np"` indicates that the return format is numpy array. Besides `np`, `return_tensors` can take the value of `tf` or `pt`, where `tf` returns Tensorflow `tf.constant` object and `pt` returns PyTorch `torch.tensor` object. If not set, it returns a list of python integers.
* `padding` means adding zeros to shorter reviews in the dataset. The `padding` argument controls how `padding` is implemented.  
 * `padding=True` is the same as `padding='longest'`. It checks the longest sequence in the batch and pads zeros to that length. There is no padding if only one text document is provided.
 * `padding='max_length'` pads to `max_length` if it is specified, otherwise, it pads to the maximum acceptable input length for the model.
 * `padding=False` is the same as `padding='do_not_pad'`. It is the default, indicating that no padding is applied, so it can output a batch with sequences of different lengths.

The labels for the reviews are converted to one-dimensional numpy arrays.

In [7]:
# Tokenizer from a pretrained model
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Tokenize the reviews
tokenized_data_train = tokenizer(X_train.to_list(), return_tensors="np", padding=True)
tokenized_data_test = tokenizer(X_test.to_list(), return_tensors="np", padding=True)

# Labels are one-dimensional numpy or tensorflow array of integers
labels_train = np.array(y_train)
labels_test = np.array(y_test)

# Tokenized ids
print(tokenized_data_train["input_ids"][0])

[  101 17554   112   189  2080  2965   119   102     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0]


After printing out the tokenized IDs for the first review, we can see that the tokenized words are converted into integers, and the sentence is padded with zeros at the end of the review.

There are two special tokens in the token IDs, `101` at the beginning of the sentence and `102` at the end of the sentence. The BERT tokenizer uses `101` to encode the special token [CLS] and `102` to encode the special token [SEP], but the other models may use other special tokens.

NOw we will build a customized transfer learning model for sentiment analysis.

* `TFAutoModelForSequenceClassification` loads the BERT model without the sequence classification head.
* The method `from_pretrained()` loads the weights from the pretrained model into the new model, so the weights in the new model are not randomly initialized. Note that the new weights for the new sequence classification head are going to be randomly initialized.
* `bert-base-cased` is the name of the pretrained model. We can change it to a different model based on the nature of the project.
* `num_labels` indicates the number of classes. Our dataset has two classes, positive and negative, so `num_labels=2`.

In [8]:
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


After loading the pretrained model, we will compile the model.
* `SparseCategoricalCrossentropy` is used as the loss function, but the Hugging Face documentation mentioned that Hugging Face models automatically choose a loss that is appropriate for their task and model architecture if the loss is not explicitly specified.
* `from_logits=True` informs the loss function that the output values are logits before applying softmax, so the values do not represent probabilities.
* We are using Adam as the optimizer and the number `5e-6` is the learning rate. A smaller learning rate corresponds to a more stable weights value update and a slower training process.
* `accuracy` is used as the metrics because we have a balanced dataset.

In [9]:
# Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile model
model.compile(optimizer=Adam(5e-6), loss=loss, metrics=['accuracy'])

In [10]:
# Fit the model
model.fit(dict(tokenized_data_train),
          labels_train,
          validation_data=(dict(tokenized_data_test), labels_test),
          batch_size=10,
          epochs=5)

Epoch 1/5
80/80 [==============================] - 655s 8s/step - loss: 0.6801 - accuracy: 0.5650 - val_loss: 0.6004 - val_accuracy: 0.7500
Epoch 2/5
80/80 [==============================] - 625s 8s/step - loss: 0.4180 - accuracy: 0.8662 - val_loss: 0.2357 - val_accuracy: 0.9200
Epoch 3/5
80/80 [==============================] - 626s 8s/step - loss: 0.1669 - accuracy: 0.9500 - val_loss: 0.2127 - val_accuracy: 0.9200
Epoch 4/5
80/80 [==============================] - 620s 8s/step - loss: 0.0793 - accuracy: 0.9762 - val_loss: 0.2297 - val_accuracy: 0.9150
Epoch 5/5
80/80 [==============================] - 609s 8s/step - loss: 0.0434 - accuracy: 0.9937 - val_loss: 0.2589 - val_accuracy: 0.9250


we will talk about how to fit a transfer learning model using Tensorflow Keras.

When fitting the model, we convert the tokenized dataset into a dictionary for Keras. `batch_size=10` means that four reviews are processed for each weights and bias update. `epochs=5` means that the model fitting process will go through the training dataset 5 times.

Accuracy after 5 sessions= 92%

All the weights will be updated by default for the transfer learning model.

If we would like to keep the pretrained model weights as is and only update the weights and bias of the output layer, we can use model.layers[0].trainable = False to freeze the weights of the BERT model.

If we would like to keep the weights of some layers and update others, we can use model.bert.encoder.layer[i].trainable = False to freeze the weights of the corresponding layers.

In general, if the dataset for the transfer learning model is large, it is suggested to update all weights, and if the dataset for the transfer learning model is small, it is suggested to freeze the pretrained model weights.

But we can always compare the model performance by adding the tunable pretrained model layers one by one.


Model prediction and evaluation for sentiment analysis transfer learning.

Passing the tokenized text to the `.predict` method, we get the predictions for the customized transfer learning sentiment model. `logits` is the last layer of the neural network before softmax is applied.

We can see that the prediction has two columns. The first column is the predicted logit for label 0 and the second column is the predicted logit for label 1. logit values do not sum up to 1.

In [11]:
# Predictions
y_test_predict = model.predict(dict(tokenized_data_test))['logits']

# First 5 predictions
y_test_predict[:5]

7/7 [==============================] - 36s 4s/step


array([[-2.920221 ,  2.0627341],
       [-2.3089607,  1.4943981],
       [-3.1491482,  2.2916358],
       [ 2.926933 , -2.316748 ],
       [-3.1631732,  2.181807 ]], dtype=float32)

To get the predicted probabilities, we need to apply softmax on the predicted logit values.

After applying softmax, we can see that the predicted probability for each review sums up to 1.

In [12]:
# Predicted probabilities and apply softmax
y_test_probabilities = tf.nn.softmax(y_test_predict)

# First 5 predicted probabilities
y_test_probabilities[:5]

<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[0.00680712, 0.99319273],
       [0.0218095 , 0.97819054],
       [0.00431736, 0.9956826 ],
       [0.994747  , 0.00525304],
       [0.00474938, 0.99525064]], dtype=float32)>

To get the predicted labels, `argmax` is used to return the index of the maximum probability for each review, which corresponds to the labels of zeros and ones.

In [13]:
# Predicted label
y_test_class_preds = np.argmax(y_test_probabilities, axis=1)

# First 5 predicted labels
y_test_class_preds[:5]

array([1, 1, 1, 0, 1])

In [14]:
# Accuracy
accuracy_score(y_test_class_preds, y_test)

0.925

In [15]:
# Save tokenizer
tokenizer.save_pretrained('./sentiment_transfer_learning_tensorflow/')

# Save model
model.save_pretrained('./sentiment_transfer_learning_tensorflow/')

In [16]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("./sentiment_transfer_learning_tensorflow/")

# Load model
loaded_model = TFAutoModelForSequenceClassification.from_pretrained('./sentiment_transfer_learning_tensorflow/')

Some layers from the model checkpoint at ./sentiment_transfer_learning_tensorflow/ were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at ./sentiment_transfer_learning_tensorflow/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [17]:
# Predict logit using the loaded model
y_test_predict = loaded_model.predict(dict(tokenized_data_test))['logits']

# Take a look at the first 5 predictions
y_test_predict[:5]

7/7 [==============================] - 37s 5s/step


array([[-2.920221 ,  2.0627341],
       [-2.3089607,  1.4943981],
       [-3.1491482,  2.2916358],
       [ 2.926933 , -2.316748 ],
       [-3.1631732,  2.181807 ]], dtype=float32)

## Sentiment Model Using Transfer Learning on Large Dataset

In [18]:
# Convert pyhton dataframe to Hugging Face arrow dataset
hg_amz_review = Dataset.from_pandas(amz_review)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Funtion to tokenize data
def tokenize_dataset(data):
    return tokenizer(data["review"])

# Tokenize the dataset
dataset = hg_amz_review.map(tokenize_dataset)

# Load model
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased")

# TF dataset
tf_dataset = model.prepare_tf_dataset(dataset=dataset,
                                      batch_size=16,
                                      shuffle=True,
                                      tokenizer=tokenizer)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [20]:
# Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile model
model.compile(optimizer=Adam(5e-6), loss=loss, metrics=['accuracy'])

# Fit the model
model.fit(tf_dataset,
          epochs=5)

Epoch 1/5
62/62 [==============================] - 491s 7s/step - loss: 0.6677 - accuracy: 0.6260
Epoch 2/5
62/62 [==============================] - 455s 7s/step - loss: 0.4221 - accuracy: 0.8730
Epoch 3/5
62/62 [==============================] - 453s 7s/step - loss: 0.2013 - accuracy: 0.9476
Epoch 4/5
62/62 [==============================] - 448s 7s/step - loss: 0.1100 - accuracy: 0.9758
Epoch 5/5
62/62 [==============================] - 451s 7s/step - loss: 0.0674 - accuracy: 0.9889
